In [58]:
import pandas as pd
import requests
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': 'BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4'}
    request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        print(request.reason)
        raise Exception('Query failed and return code is {}.{}'.format(request.status_code, query))

# Get all Instructions associated with Friktion Volt Program

In [360]:
query = """
query MyQuery {
  solana {
    instructions(
      date: {between: ["2022-03-01", "2022-03-15"]}
      success: {is: true}
      programId: {is: "VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp"}
    ) {
      action {
        name
      }
      block {
        timestamp {
          iso8601
        }
      }
      callPath
      log {
        consumed
        instruction
        logs
      }
      transaction {
        signature
        success
        feePayer
      }
    }
  }
}

"""

result = run_query(query)
# convert GraphQL json to pandas dataframe
df = pd.json_normalize(result['data']['solana']['instructions'])

## Group Instructions by Type

In [361]:
df.groupby("log.instruction")["transaction.success"].count().sort_values()[::-1].head(20)

log.instruction
                                        3060
Deposit                                 1156
ClaimPending                             630
Withdraw                                 539
ClaimPendingWithdrawal                   468
CancelPendingDeposit                      68
InitNewAccounts                           34
RebalanceEnter                            33
SettleEnterFunds                          33
CancelPendingWithdrawal                   27
SetNextOption                             20
RebalancePrepare                          20
InitSerumMarket                           20
StartRound                                19
EndRound                                  15
RebalanceSettle                           15
TakePendingWithdrawalFees                 15
SettlePermissionedMarketPremiumFunds      14
RebalanceSwapPremium                      12
SettleSwapPremiumFunds                    10
Name: transaction.success, dtype: int64

In [362]:
df.columns

Index(['callPath', 'action.name', 'block.timestamp.iso8601', 'log.consumed',
       'log.instruction', 'log.logs', 'transaction.signature',
       'transaction.success', 'transaction.feePayer'],
      dtype='object')

In [363]:
df.loc[df["transaction.feePayer"]=="sx4c5i2XbiSJ2uYG72m4w131q7tTdyfAdAhSST3Wx57"]

,callPath,action.name,block.timestamp.iso8601,log.consumed,log.instruction,log.logs,transaction.signature,transaction.success,transaction.feePayer
3032,0,,2022-03-06T15:22:15Z,60196,ClaimPendingWithdrawal,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: ClaimPendingWithdrawal;Program log: validating ClaimPendingWithdrawal context;Program log: claim_pending withdrawal...;Program log: transferring w/ seeds 100247 tokens from hBkJbNrxGMYb9juyvoS2KdmGXQcds73fA2BB4VvY7uU to 2YvimTxp9X7V22ZC3Xe4QfwdaZLFDBg1qe9yQ7bH9nG9;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 60196 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,2nwd9uJyP51G5zvoqPtY5VBpXb76cdXAKnmEwF2xYPwi26yGhcnX4XRMtz2DWgauQZQQ1G123om7SNFVpk1tyQtE,True,sx4c5i2XbiSJ2uYG72m4w131q7tTdyfAdAhSST3Wx57
3034,0,,2022-03-06T15:23:18Z,88786,Deposit,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program log: validating Deposit context;Program log: beginning deposit...;Program log: user underlying = 25100247, deposit amount = 25100247, vault capacity = 150000000000;Program log: post deposit estimated underlying: 120048710239, vault capacity: 150000000000;Program log: pending deposit;Program log: transferring 25100247 tokens from 2YvimTxp9X7V22ZC3Xe4QfwdaZLFDBg1qe9yQ7bH9nG9 to GVViqUdyWWjKRM29VG7rwBshkcaRcoiurAJp8TZCCiaz;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 88786 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",21XbkC9jUr1hH7mcT9uxtVhuv3mG1gagDhPYuZxydvCSGSuB7a7abNTnsw7XxR236eSiurZfHhXeGCnBTfKR2pv2,True,sx4c5i2XbiSJ2uYG72m4w131q7tTdyfAdAhSST3Wx57


In [250]:
df.loc[df["transaction.feePayer"]=="6SvTZfYs53eUpvYWnywdqzgud5x5atzubPhns82Aa3R7"].sort_values("block.timestamp.iso8601")

,callPath,action.name,block.timestamp.iso8601,log.consumed,log.instruction,log.logs,transaction.signature,transaction.success,transaction.feePayer
3,0,,2022-03-20T00:43:45Z,56761,ClaimPending,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: ClaimPending;Program log: validating ClaimPending context;Program log: claim_pending deposit...;Program log: transferring w/ seeds 1377609104 tokens from C4KyYFoxLaLHV8We12EEQATViWRmq5WV4m4pYm1Ce29e to 8cErNBW6D9ni7Djx5xTmcK9UJ7ec4EJSA1jPHy94HyQs;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 56761 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,3xhBR9QuzVswAqUzRAsqyVUBoKtut41A336H1TLacVAWYghCVEUo6hjGRW1qCCJeTkeUGVw6aXEGo77fHDkmr5dA,True,6SvTZfYs53eUpvYWnywdqzgud5x5atzubPhns82Aa3R7
4,3,,2022-03-20T00:43:45Z,86140,Deposit,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program log: validating Deposit context;Program log: beginning deposit...;Program log: user underlying = 1250000000, deposit amount = 1250000000, vault capacity = 50000000000000;Program log: post deposit estimated underlying: 13131000087165, vault capacity: 50000000000000;Program log: pending deposit;Program log: transferring 1250000000 tokens from HkwZD4Xps1nVSbe6kK66kwrHBHLh7DfAuMQyX8Amo3az to FnCg7sLfe8gz4UyrmN9c3VX6Axohu1Pa8wy3bsSPZwfq;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 86140 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",3xhBR9QuzVswAqUzRAsqyVUBoKtut41A336H1TLacVAWYghCVEUo6hjGRW1qCCJeTkeUGVw6aXEGo77fHDkmr5dA,True,6SvTZfYs53eUpvYWnywdqzgud5x5atzubPhns82Aa3R7
5,2,,2022-03-20T00:44:49Z,86146,Deposit,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program log: validating Deposit context;Program log: beginning deposit...;Program log: user underlying = 1250000000, deposit amount = 1250000000, vault capacity = 50000000000000;Program log: post deposit estimated underlying: 13132250087165, vault capacity: 50000000000000;Program log: pending deposit;Program log: transferring 1250000000 tokens from HaJM4P7FWbijgJ8K81sLSsemNAwpWDRZsNQfcjgbZNax to FnCg7sLfe8gz4UyrmN9c3VX6Axohu1Pa8wy3bsSPZwfq;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 86146 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",2JHPPfiPxEYNM2dg2oWBcGivRtbCWsmGEJpwAoGCo1ayuygWmZ5TL6FrPDZgrzTWNPiRXgGSCiAgyBNW1bDCLAvS,True,6SvTZfYs53eUpvYWnywdqzgud5x5atzubPhns82Aa3R7
64,2,,2022-03-20T04:07:56Z,86002,Deposit,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program log: validating Deposit context;Program log: beginning deposit...;Program log: user underlying = 1000000000, deposit amount = 1000000000, vault capacity = 50000000000000;Program log: post deposit estimated underlying: 13133250087165, vault capacity: 50000000000000;Program log: pending deposit;Program log: transferring 1000000000 tokens from 4rkh9azYtQwmmx4DtdKxYs9Ryka9vzkWUEyL177uJvob to FnCg7sLfe8gz4UyrmN9c3VX6Axohu1Pa8wy3bsSPZwfq;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 86002 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",2mJK3FRn9ufX5tZnUeqLmZZEv8N7iZSJ1odAiZzY7JSKnHkxzpyiDjQKxi4s7Lh56ZiN4xWHLXwpHFsHSGHPf4vE,True,6SvTZfYs53eUpvYWnywdqzgud5x5atzubPhns82Aa3R7
254,2,,2022-03-20T16:47:55Z,86014,Deposit,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program log: validating Deposit context;Program log: beginning deposit...;Program log: user underlying = 500000000, deposit amount = 500000000, vault capacity = 50000000000000;Program log: post deposit estimated underlying: 13147396087165, vault capacity: 50000000000000;Program log: pending deposit;Program log: transferring 500000000 tokens from 6MtLJynGePZXsNN8VcUp13Tt5SE9gwYnQjQKQ2WxD3qd to FnCg7sLfe8gz4UyrmN9c3VX6Ax

In [251]:
df.loc[df["log.instruction"]=="CancelPendingWithdrawal"].sort_values("block.timestamp.iso8601")

,callPath,action.name,block.timestamp.iso8601,log.consumed,log.instruction,log.logs,transaction.signature,transaction.success,transaction.feePayer
87,0,,2022-03-20T05:52:03Z,38195,CancelPendingWithdrawal,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: CancelPendingWithdrawal;Program log: validating CancelPendingWithdrawal context;Program log: beginning cancel pending withdraw...;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 38195 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,2dpcxesoJjb6kgfeFsTR1iiLV2VTVWxcYjtH1vr7GF7a73BaWxnoV1QidBUjrNW7NEZe6HhChiQvJMpwdLT1hzyL,True,36EJ19QKSj8UQnThx7wAZ73EtbwcWSuwAFspBBrfd3nL
207,0,,2022-03-20T13:49:50Z,33695,CancelPendingWithdrawal,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: CancelPendingWithdrawal;Program log: validating CancelPendingWithdrawal context;Program log: beginning cancel pending withdraw...;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 33695 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,355f9uxDuoBMN789JYS8cLVJArWeNe91cZVdFKcQ2fBawenvuvxZmYcCmjwjwaJS7guiEvxhMJcpSh6NNhf8Tv9A,True,9DzL5aYgerzgfe4bH8UfADbv9afhLs4BLNb5nQqXSLwR
312,0,,2022-03-20T23:00:25Z,32195,CancelPendingWithdrawal,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: CancelPendingWithdrawal;Program log: validating CancelPendingWithdrawal context;Program log: beginning cancel pending withdraw...;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 32195 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,4g3ADnEtiTzL5UTym1YP1hHR62xihoWeR9iV43NksRCE34WAkMvwwUgXBzTfSkD562nSkpzh4scWP5SNwpwwFmSP,True,8Aub41vv431d7hh4P6pPpnegBRyQ8uBRo8RD4KaU7REU
367,0,,2022-03-21T04:01:09Z,41195,CancelPendingWithdrawal,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: CancelPendingWithdrawal;Program log: validating CancelPendingWithdrawal context;Program log: beginning cancel pending withdraw...;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 41195 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,5bTMj8HQALt3Y8D3ypsSt9BVwZMNaD6QhJJKmVUJRy8jtooFYdegQZzkpjnaKbYvkkSj48SvFnsG2WB61jbmeSn6,True,XR1SvrmTud7MzwS1DtiULLUnvuHHdegYM8MJmHUC2SZ
372,0,,2022-03-21T04:32:13Z,41195,CancelPendingWithdrawal,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: CancelPendingWithdrawal;Program log: validating CancelPendingWithdrawal context;Program log: beginning cancel pending withdraw...;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 41195 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,3dRuiGjrcbf2meGRHsL14pbCZMCZxF2QtYrmQg2cAmPbndF1BGG9WGBmkTh6bCuWi616gJVPHNJQpCtQtpy1Ecpo,True,FXa8unfp5phYpeeqHYzUwCQSGnforFgD3E3gps75oRPv
604,0,,2022-03-21T20:19:43Z,42695,CancelPendingWithdrawal,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: CancelPendingWithdrawal;Program log: validating CancelPendingWithdrawal context;Program log: beginning cancel pending withdraw...;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 42695 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,3F5K2Jgj4g9NeP3NACGYin6zz3UEnT5g5HnV9WMtrMsuo1gwotkrCrvwmWQgLHxPHAF3QcXu4BMj7or19tYH8rT5,True,EDvWBmYG1fBFy4aGuo4kn42R9WeUAcxUU1T6GFjVcj4K
672,0,,2022-03-22T00:40:18Z,36695,CancelPendingWithdrawal,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: CancelPendingWithdrawal;Program log: validating CancelPendingWithdrawal context;Program log: beginning cancel pending withdraw...;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 36695 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,3MCfGkUSnwNhJZ6C2Ev7zKwyB

In [320]:
    df.loc[df["log.instruction"]=="EndRound"]

,callPath,action.name,block.timestamp.iso8601,log.consumed,log.instruction,log.logs,transaction.signature,transaction.success,transaction.feePayer
1527,0,,2022-03-24T02:28:07Z,158689,EndRound,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: EndRound;Program log: validating EndRound context;Program log: ending round...;Program log: 1648088887;Program log: claim performance fees...;Program log: underlying pre enter = 10023035934485, post settle = 10056455341373;Program log: epoch pnl = 33419406888, performance fee = 3341940688;Program log: transferring w/ seeds 3341940688 tokens from 3C8SvZ9EcYNbEesWh5nF7hEBZgx632ntvkZk5MQ8Hk6h to 4gQT3fvf45bQhvRW1CDKdovVkuyEMggLg4fwXoHtLFqY;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 158689 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",4Dh7N3Ta3PeGLer3W9xHCmq63rwR89ZQBNuzxRUYrYd3HhuCq6ws3PuNmqjDviHX413TMQ1teVHuBjao9WhVdj37,True,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE
1535,0,,2022-03-24T02:34:52Z,162971,EndRound,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: EndRound;Program log: validating EndRound context;Program log: ending round...;Program log: 1648089292;Program log: claim performance fees...;Program log: underlying pre enter = 1000100000000, post settle = 1006262534000;Program log: epoch pnl = 6162534000, performance fee = 616253400;Program log: transferring w/ seeds 616253400 tokens from P7TTBhKxrQg4BS651fB5cLX9Jn2UyL2LxeNayrLEMbX to 4gQT3fvf45bQhvRW1CDKdovVkuyEMggLg4fwXoHtLFqY;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 162971 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",5a2wVzyBr9fU8pECQAfNNEknqwoqysFvgprcQWYn7v5K1efbunG24Cbiq2voZSYZKqeagnALhL2mv1iqwfzrgPxs,True,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE
2324,0,,2022-03-25T02:31:53Z,165403,EndRound,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: EndRound;Program log: validating EndRound context;Program log: ending round...;Program log: 1648175513;Program log: claim performance fees...;Program log: underlying pre enter = 215796691226199, post settle = 216695311226199;Program log: epoch pnl = 898620000000, performance fee = 89862000000;Program log: transferring w/ seeds 89862000000 tokens from unrWTSSfz6H7DzYYHRTuTCK3yVzR1YUNqUn7xWwr4Z5 to 5byrk6XpxfiCsWPdVhQR9csGUBdmiCwLZCWM5AY7kbq2;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 165403 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",yUaML9jVW8nABTD6PjeWAXykt55SjtXdNdrFycgvq463SyKra4S4qs6tYNF6VYH7Z91NxoMcujboWrxFcUH6J6Z,True,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE
2396,0,,2022-03-25T02:52:56Z,133297,EndRound,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: EndRound;Program log: validating EndRound context;Program log: ending round...;Program log: 1648176776;Program log: claim performance fees...;Program log: underlying pre enter = 11639746626986, post settle = 11348708605700;Program log: managing underlying for pending withdrawals...;Program log: pending withdrawal fees = 0;Program log: transferring w/ seeds 3254911650376 tokens from 9gQVba4aPF6nc2Tkv3BZmWrerJRmDSwxpTB6bfAiV97Q to H5Py6V5ZyFDZkWM8Ab1TngCQdJxRd8H2m43ooK7yYKLM;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 133297 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",3XvpwAdXMKTbYU57FqaKrTwZPWkgV1DJ2pdhmfgcbsxdVwSiHGowB78uX2fdkBor1ktouKE1XjJryJmMq9MfPn2m,True,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE
2414,0,,2022-03-25T02:58:17Z,161661,EndRound,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: EndRound;Program log: validating EndRound context;Program log: ending round...;Program log: 1648177097;Program log: claim performance fees...;Program log: underlying pre enter = 1374753973853, post settle = 13866

## How to calculate
Deposits = create account - close account
Cancel Pending Deposits = close Account - initialize account

Withdrawal = Burn
Cancel Pending Withdrawal = Mint

# Deposit Demo

In [307]:
# Format this shit so it plugs into the query

tx_signatures = list(df.loc[
    df["log.instruction"]=="Deposit"]["transaction.signature"].iloc[50:100].values)

tx_signatures = str(tx_signatures)[1:-1].replace("\'", "\"").replace("\n", "")

In [308]:
query = """
query MyQuery {
  solana(network: solana) {
    transfers(
      signature: {in: [""" + tx_signatures + """]}
    ) {
      instruction {
        action {
          name
          type
        }
        callPath
      }
      amount(success: {is: true})
      transaction {
        signer
        signature
      }
      block {
        timestamp {
          iso8601
        }
      }
      currency {
        name
        address
        symbol
        decimals
      }
      sender {
        address
        mintAccount
        type
      }
      receiver {
        address
        mintAccount
        type
      }
    }
  }
}
"""

result = run_query(query)
# convert GraphQL json to pandas dataframe
deposits = pd.json_normalize(result['data']['solana']['transfers'])

In [309]:
deposits = deposits.loc[deposits["currency.name"]!="Wrapped SOL"].sort_values("instruction.callPath")

In [314]:
deposits

,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,transaction.signature,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type
50,10.002039,createAccount,createAccount,0,CtTu7Sm23a5AoNw4Wy56xcpizxUQKh7nBfr2GmU6CiG2,498DALm665TiqNFvP7HLQ3EMTw7bNrFn3DH5qWjhD2rEBje1kFfzqtqJU1qSTZSq9BkepC83Wk29QFQ4YZ4egVQL,2022-03-20T10:35:56Z,Solana,-,SOL,9,CtTu7Sm23a5AoNw4Wy56xcpizxUQKh7nBfr2GmU6CiG2,CtTu7Sm23a5AoNw4Wy56xcpizxUQKh7nBfr2GmU6CiG2,account,63Z9fAyAaTKPriWoMq3rzAi4cp6Rp1EVoAUXNLqhpvF,63Z9fAyAaTKPriWoMq3rzAi4cp6Rp1EVoAUXNLqhpvF,account
48,0.524039,createAccount,createAccount,0,8ccE2LNheWt8xyVJPvWgue3KCdqiED5HZ4YqEeGiyjMP,5ZECbWCa54crrpKdmSxZdo9yQAuYAc5ruyJYmpvRsUZHbDqm9CM9Bfe4HC1Jt76jBWNzDrfxFkdHW3XXEBYno7Si,2022-03-20T16:53:21Z,Solana,-,SOL,9,8ccE2LNheWt8xyVJPvWgue3KCdqiED5HZ4YqEeGiyjMP,8ccE2LNheWt8xyVJPvWgue3KCdqiED5HZ4YqEeGiyjMP,account,DgW6SaUzf5U1W5PKL5AeCtyugJEb7ssQyC4WtcnsF3mB,DgW6SaUzf5U1W5PKL5AeCtyugJEb7ssQyC4WtcnsF3mB,account
47,1.748039,createAccount,createAccount,0,8ABE9RWSz6FeETRMrdAWye2euXujZgyKm77eaqBkgBpD,5SMpgj93T4bEAK7feRH7wRYhnDGckG8Z87oiKX5WTYd6DofrVCo1KgWtaXLuPYqDfppAPi75iP5qLi3RtLP1BJgx,2022-03-20T11:07:58Z,Solana,-,SOL,9,8ABE9RWSz6FeETRMrdAWye2euXujZgyKm77eaqBkgBpD,8ABE9RWSz6FeETRMrdAWye2euXujZgyKm77eaqBkgBpD,account,6agp3YimY5E6oqSdfsB1QtkbMXryr9JG1u6QUyzQpxT5,6agp3YimY5E6oqSdfsB1QtkbMXryr9JG1u6QUyzQpxT5,account
46,0.502039,createAccount,createAccount,0,6SvTZfYs53eUpvYWnywdqzgud5x5atzubPhns82Aa3R7,5hVGT9szdRmzH2BWbgumqWVjDKc5g35oX5BX6mx5BZjGVDVDbm2FCttrMfsgM2WR22CPrgDj9zcsGSQqX6sU6dt,2022-03-20T16:47:55Z,Solana,-,SOL,9,6SvTZfYs53eUpvYWnywdqzgud5x5atzubPhns82Aa3R7,6SvTZfYs53eUpvYWnywdqzgud5x5atzubPhns82Aa3R7,account,6MtLJynGePZXsNN8VcUp13Tt5SE9gwYnQjQKQ2WxD3qd,6MtLJynGePZXsNN8VcUp13Tt5SE9gwYnQjQKQ2WxD3qd,account
45,10.002039,createAccount,createAccount,0,4hATAoKD9MRXdqAxBcLYzeMX4kc54aW41Z7AaoEsc3c4,3AjWkBroY5R3okb8cojHJrBHBBE6RQ2FJaMAEYssPaMdYEQuKLXm6gkDdw4EsUW3aZy3pU2u97VGYDPotJJrWdv6,2022-03-20T10:59:45Z,Solana,-,SOL,9,4hATAoKD9MRXdqAxBcLYzeMX4kc54aW41Z7AaoEsc3c4,4hATAoKD9MRXdqAxBcLYzeMX4kc54aW41Z7AaoEsc3c4,account,FKsr7H9eHEHUdbCNyLLvCDEWpD86yP6NsJE4naKvKPex,FKsr7H9eHEHUdbCNyLLvCDEWpD86yP6NsJE4naKvKPex,account
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,0.002039,closeAccount,closeAccount,4,8ABE9RWSz6FeETRMrdAWye2euXujZgyKm77eaqBkgBpD,5SMpgj93T4bEAK7feRH7wRYhnDGckG8Z87oiKX5WTYd6DofrVCo1KgWtaXLuPYqDfppAPi75iP5qLi3RtLP1BJgx,2022-03-20T11:07:58Z,Solana,-,SOL,9,6agp3YimY5E6oqSdfsB1QtkbMXryr9JG1u6QUyzQpxT5,6agp3YimY5E6oqSdfsB1QtkbMXryr9JG1u6QUyzQpxT5,account,8ABE9RWSz6FeETRMrdAWye2euXujZgyKm77eaqBkgBpD,8ABE9RWSz6FeETRMrdAWye2euXujZgyKm77eaqBkgBpD,account
14,0.002039,closeAccount,closeAccount,4,3jJBxEujKsutB7v8z1bUUAQjoY1VwoKotmYxLR7TEZzw,SbwGzVFnJDZakRVhmHxpn3zbLY3oN2dhRQqFGaJpJw4DBhNHHe2dmNMsxS4baiA4HRzT43NRxi1b2znhzLX6KfX,2022-03-20T11:32:23Z,Solana,-,SOL,9,DaMPc1v92XEENwpAn5vTqevugCXSGnFjuECLWjAxVyfe,DaMPc1v92XEENwpAn5vTqevugCXSGnFjuECLWjAxVyfe,account,3jJBxEujKsutB7v8z1bUUAQjoY1VwoKotmYxLR7TEZzw,3jJBxEujKsutB7v8z1bUUAQjoY1VwoKotmYxLR7TEZzw,account
12,0.002039,closeAccount,closeAccount,4,3d1Botrw38mDR9LpbtrgvVZD8cNZb3WNKRWn4uj7LHVx,3mGeM91QdA9h2cFypRZ8HVBdewYhVwSS6XqLHj2pRML4AQ32ToMAf5uiddtMYmEmxkp8Prvutx954ue6ugDoFQ5,2022-03-20T14:33:24Z,Solana,-,SOL,9,5cXXa4Fmcfa5BWo3TGUiS48e6zSM2FC2Qetn2GY1cU4,5cXXa4Fmcfa5BWo3TGUiS48e6zSM2FC2Qetn2GY1cU4,account,3d1Botrw38mDR9LpbtrgvVZD8cNZb3WNKRWn4uj7LHVx,3d1Botrw38mDR9LpbtrgvVZD8cNZb3WNKRWn4uj7LHVx,account
8,0.002039,closeAccount,closeAccount,4,32ByKf8vuiaBzLaY5gN6SD3cPTqG6AzJeCvW8xZwwHbr,2Vod2nnUybR2eopLZRyrQcKie9KPPcMYqUpn7ee2UkAE4koWLbPvq6zbnJq4xVTm2VY8xG6tGDpEDfCiA3noG3LT,2022-03-20T18:14:06Z,Solana,-,SOL,9,8jzdFPNwje64svdE7CxXusMzYtbuWt9hsd7cmRxNVfLJ,8jzdFPNwje64svdE7CxXusMzYtbuWt9hsd7cmRxNVfLJ,account,32ByKf8vuiaBzLaY5gN6SD3cPTqG6AzJeCvW8xZwwHbr,32ByKf8vuiaBzLaY5g

In [315]:
# def get_transfer_fields(x):
#     print(x)
#     lastrow = x.loc[x["instruction.action.name"]=="transfer"].sort_values("instructino.callPath")[-1] 
#     return lastrow[["amount", "currency.name", "receiver.mintAccount"]]
final = deposits.loc[deposits["instruction.action.name"]=="transfer"].groupby("transaction.signature").last().reset_index()

In [317]:
final.to_csv("./sample_deposits.csv")

# Cancel Pending Withdrawal Demo

In [263]:
# Format this shit so it plugs into the query

tx_signatures = list(df.loc[
    df["log.instruction"]=="CancelPendingWithdrawal"]["transaction.signature"].iloc[50:100].values)

tx_signatures = str(tx_signatures)[1:-1].replace("\'", "\"").replace("\n", "")

In [270]:
query = """
query MyQuery {
  solana(network: solana) {
    transfers(
      signature: {in: [""" + tx_signatures + """]}
    ) {
      instruction {
        action {
          name
          type
        }
        callPath
      }
      amount(success: {is: true})
      transaction {
        signer
        signature
      }
      block {
        timestamp {
          iso8601
        }
      }
      currency {
        name
        address
        symbol
        decimals
      }
      sender {
        address
        mintAccount
        type
      }
      receiver {
        address
        mintAccount
        type
      }
    }
  }
}
"""

result = run_query(query)
# convert GraphQL json to pandas dataframe
cancel_withdrawals = pd.json_normalize(result['data']['solana']['transfers'])

In [271]:
cancel_withdrawals = cancel_withdrawals.loc[cancel_withdrawals["currency.name"]!="Wrapped SOL"].sort_values("instruction.callPath")

In [279]:
cancel_withdrawals

,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,transaction.signature,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type
0,265.313148,mintTo,mintTo,0-0,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,5THSMzhv6NAkNRLSY2LDAfAE8nEx7rycV5QnEpKMs1TkjpVN5jvwGU5NRQgcHHVorFF4MRtSDq673HpGaDQZQC9x,2022-03-27T16:54:27Z,,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,,9,,,,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,FzBVYYkYRtsWWSTNmxdrx8iKZdCcBUGnMGsoowg6JCvJ,account
1,3.713803,mintTo,mintTo,0-0,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,VFA1SJ2iV96njG8RdpeL9eMe78kM3ju5MAsnDDCqpYZwdNuSwMLw8EZv7pN4DZouGzWpVX4KFfBTepGcM6KJhqc,2022-03-27T16:59:18Z,,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,,9,,,,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,FzBVYYkYRtsWWSTNmxdrx8iKZdCcBUGnMGsoowg6JCvJ,account
2,4805.700237,mintTo,mintTo,0-0,42i4VhjfjDv1DYdSfGsZXW53eGqFhBdWU6Lg6FcoPNg8,4Aa1mQ119X8epmyp3jNYFN4mgKJGrR1b7V9B3g29TkLBvukmAQo3N6jCUscj1YYfR4ZFRXcWjYhoTx8VTz5NJJz8,2022-03-28T09:46:58Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,,,,42i4VhjfjDv1DYdSfGsZXW53eGqFhBdWU6Lg6FcoPNg8,D2aJY1GDWhVzZQitbWUbWzsvc5oVvXr371CdUJiJnHKu,account
3,277.191034,mintTo,mintTo,0-0,7etxrP9LU9xwDVfGpEUrG1XYhq64ttE5rbrGhGymQrvY,hx2LKHaY9CSr6B4ayYKLFiyjZj2isjRfBjTzfN1nEwZrqEbRW7DqM9q8aHYTjQ6mUceeG45AskXtzuDABDTqYzV,2022-03-28T10:49:01Z,,Fw8mw9zNyfngvBYuS2L6XWfvFE8fExQ3apZVw4Ery5oy,,6,,,,7etxrP9LU9xwDVfGpEUrG1XYhq64ttE5rbrGhGymQrvY,3UXf3Sb1fVMhNCPNgTbn2uGL37iY2EryHdqhuEP7cjKv,account
4,6.702134,mintTo,mintTo,0-0,8cSDdsAe4kBxfTD28H2HRzT7Hhp5DZxihRZGL2Lpmwrt,3nCnNMb3q9j67HnpyG7ZKEd2AADMRK2n8FvysGcrMMSy5mSTsAZSGN5quwgYPsyqg2E1DnbRxEVWUcTQh8W73e6e,2022-03-28T11:37:57Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,,,,8cSDdsAe4kBxfTD28H2HRzT7Hhp5DZxihRZGL2Lpmwrt,5NjpSprzTR3c1WiLHaCUrbYvP1wKykL5sMveUc9ALo29,account
5,5204.693398,mintTo,mintTo,0-0,BiEXHzcCgTgMLEPWVMJqCT38z1V1Xo3oxc7qxp2bUDUZ,ENWyWi7mjDon7AjAeDnEAB9zXwceEVdE2v53J7wN7ANQrcpBzkDMdWLhhHXApqxxUgBU6sTRciuSXVAqitp7tDk,2022-03-27T21:48:16Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,,,,BiEXHzcCgTgMLEPWVMJqCT38z1V1Xo3oxc7qxp2bUDUZ,EHsEiE5U4gMmPc5NEGtx1jqW4crWYvKKrzYY1JKjf4nw,account
6,4.563915,mintTo,mintTo,0-0,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,3kKBErted5MM9C6yRkTCLFk4TL8eDCuAZybGsLUNgaog7S2F5xasbiaeMNYU4eQ1fqzva71bwiGyJQhJKQuL1qDS,2022-03-28T17:11:28Z,,5SLqZSywodLS8ih6U2AAioZrxpgR149hR8SApmCB7r5X,,6,,,,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,4ZUqHFxi7sP6BpPa28hvK8bqrk4DQJEktUzdymkySvfq,account
7,3.223691,mintTo,mintTo,0-0,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,pH5pnbCqa29bTiYuq2AQxaMyzzJZttJkQFunHGh1KKKJWq2YxfVXLyyHPnyysvUfRKBo311QLj4YVT9AwT9KTSZ,2022-03-28T17:11:22Z,,SoAnGsHVqSyaN4MjWoPCcftC1V6oSeNxLJjF5TXrtuL,,6,,,,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,ED9iWMN5fiw3Cnon46fEvZaw2CqhDpFgw5Pnyu1qvN3T,account
8,29.507184,mintTo,mintTo,0-0,EgzRJHeqbfYsCL4VBZCsq2ZEpk5M7jzWc4n2N53SyFKJ,45NPN2TTJ8FduLkwFASuebq5Jrvs2NgyR3t8KHtk4TMMj1DBJpePbEo5nFoszGetgTPTKKLSbTPM3vhZzEcFr4eG,2022-03-28T14:24:47Z,,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,,6,,,,EgzRJHeqbfYsCL4VBZCsq2ZEpk5M7jzWc4n2N53SyFKJ,DYod4eQKxy4XThto93MNGng7tXv9p7Rgyn6MDNNCg8kW,account
9,0.278102,mintTo,mintTo,0-0,Eu8S7Kt3MtfSDx4UsNh6FaBw8j5DYhC53L48f8D9R4c5,3UTfRTMFkCd3hz5UpZCp5JhM4SY8raum8yJQJAxErj7UwEbKRNzifSY7F2XdiuZihYVK9mi9hFKtWmmMUMGtA7Cx,2022-03-27T20:54:10Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,,,,Eu8S7Kt3MtfSDx4UsNh6FaBw8j5DYhC53L48f8D9R4c5,EBktJULvmhnng2zwkUwyjPqejPzxbJjyHqey7nMwQLzy,account


In [323]:
# def get_transfer_fields(x):
#     print(x)
#     lastrow = x.loc[x["instruction.action.name"]=="transfer"].sort_values("instructino.callPath")[-1] 
#     return lastrow[["amount", "currency.name", "receiver.mintAccount"]]
final = cancel_withdrawals.loc[cancel_withdrawals["instruction.action.name"]=="mintTo"].groupby("transaction.signature").last().reset_index()

In [324]:
final

,transaction.signature,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type
0,3UTfRTMFkCd3hz5UpZCp5JhM4SY8raum8yJQJAxErj7UwEbKRNzifSY7F2XdiuZihYVK9mi9hFKtWmmMUMGtA7Cx,0.278102,mintTo,mintTo,0-0,Eu8S7Kt3MtfSDx4UsNh6FaBw8j5DYhC53L48f8D9R4c5,2022-03-27T20:54:10Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,,,,Eu8S7Kt3MtfSDx4UsNh6FaBw8j5DYhC53L48f8D9R4c5,EBktJULvmhnng2zwkUwyjPqejPzxbJjyHqey7nMwQLzy,account
1,3kKBErted5MM9C6yRkTCLFk4TL8eDCuAZybGsLUNgaog7S2F5xasbiaeMNYU4eQ1fqzva71bwiGyJQhJKQuL1qDS,4.563915,mintTo,mintTo,0-0,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,2022-03-28T17:11:28Z,,5SLqZSywodLS8ih6U2AAioZrxpgR149hR8SApmCB7r5X,,6,,,,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,4ZUqHFxi7sP6BpPa28hvK8bqrk4DQJEktUzdymkySvfq,account
2,3nCnNMb3q9j67HnpyG7ZKEd2AADMRK2n8FvysGcrMMSy5mSTsAZSGN5quwgYPsyqg2E1DnbRxEVWUcTQh8W73e6e,6.702134,mintTo,mintTo,0-0,8cSDdsAe4kBxfTD28H2HRzT7Hhp5DZxihRZGL2Lpmwrt,2022-03-28T11:37:57Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,,,,8cSDdsAe4kBxfTD28H2HRzT7Hhp5DZxihRZGL2Lpmwrt,5NjpSprzTR3c1WiLHaCUrbYvP1wKykL5sMveUc9ALo29,account
3,45NPN2TTJ8FduLkwFASuebq5Jrvs2NgyR3t8KHtk4TMMj1DBJpePbEo5nFoszGetgTPTKKLSbTPM3vhZzEcFr4eG,29.507184,mintTo,mintTo,0-0,EgzRJHeqbfYsCL4VBZCsq2ZEpk5M7jzWc4n2N53SyFKJ,2022-03-28T14:24:47Z,,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,,6,,,,EgzRJHeqbfYsCL4VBZCsq2ZEpk5M7jzWc4n2N53SyFKJ,DYod4eQKxy4XThto93MNGng7tXv9p7Rgyn6MDNNCg8kW,account
4,4Aa1mQ119X8epmyp3jNYFN4mgKJGrR1b7V9B3g29TkLBvukmAQo3N6jCUscj1YYfR4ZFRXcWjYhoTx8VTz5NJJz8,4805.700237,mintTo,mintTo,0-0,42i4VhjfjDv1DYdSfGsZXW53eGqFhBdWU6Lg6FcoPNg8,2022-03-28T09:46:58Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,,,,42i4VhjfjDv1DYdSfGsZXW53eGqFhBdWU6Lg6FcoPNg8,D2aJY1GDWhVzZQitbWUbWzsvc5oVvXr371CdUJiJnHKu,account
5,5THSMzhv6NAkNRLSY2LDAfAE8nEx7rycV5QnEpKMs1TkjpVN5jvwGU5NRQgcHHVorFF4MRtSDq673HpGaDQZQC9x,265.313148,mintTo,mintTo,0-0,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,2022-03-27T16:54:27Z,,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,,9,,,,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,FzBVYYkYRtsWWSTNmxdrx8iKZdCcBUGnMGsoowg6JCvJ,account
6,ENWyWi7mjDon7AjAeDnEAB9zXwceEVdE2v53J7wN7ANQrcpBzkDMdWLhhHXApqxxUgBU6sTRciuSXVAqitp7tDk,5204.693398,mintTo,mintTo,0-0,BiEXHzcCgTgMLEPWVMJqCT38z1V1Xo3oxc7qxp2bUDUZ,2022-03-27T21:48:16Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,,,,BiEXHzcCgTgMLEPWVMJqCT38z1V1Xo3oxc7qxp2bUDUZ,EHsEiE5U4gMmPc5NEGtx1jqW4crWYvKKrzYY1JKjf4nw,account
7,VFA1SJ2iV96njG8RdpeL9eMe78kM3ju5MAsnDDCqpYZwdNuSwMLw8EZv7pN4DZouGzWpVX4KFfBTepGcM6KJhqc,3.713803,mintTo,mintTo,0-0,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,2022-03-27T16:59:18Z,,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,,9,,,,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,FzBVYYkYRtsWWSTNmxdrx8iKZdCcBUGnMGsoowg6JCvJ,account
8,hx2LKHaY9CSr6B4ayYKLFiyjZj2isjRfBjTzfN1nEwZrqEbRW7DqM9q8aHYTjQ6mUceeG45AskXtzuDABDTqYzV,277.191034,mintTo,mintTo,0-0,7etxrP9LU9xwDVfGpEUrG1XYhq64ttE5rbrGhGymQrvY,2022-03-28T10:49:01Z,,Fw8mw9zNyfngvBYuS2L6XWfvFE8fExQ3apZVw4Ery5oy,,6,,,,7etxrP9LU9xwDVfGpEUrG1XYhq64ttE5rbrGhGymQrvY,3UXf3Sb1fVMhNCPNgTbn2uGL37iY2EryHdqhuEP7cjKv,account
9,pH5pnbCqa29bTiYuq2AQxaMyzzJZttJkQFunHGh1KKKJWq2YxfVXLyyHPnyysvUfRKBo311QLj4YVT9AwT9KTSZ,3.223691,mintTo,mintTo,0-0,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,2022-03-28T17:11:22Z,,SoAnGsHVqSyaN4MjWoPCcftC1V6oSeNxLJjF5TXrtuL,,6,,,,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,ED9iWMN5fiw3Cnon46fEvZaw2CqhDpFgw5Pnyu1qvN3T,account


# Withdrawal Demo

In [354]:
# Format this shit so it plugs into the query

tx_signatures = list(df.loc[
    df["log.instruction"]=="Withdraw"]["transaction.signature"].iloc[:50].values)

tx_signatures = str(tx_signatures)[1:-1].replace("\'", "\"").replace("\n", "")

query = """
query MyQuery {
  solana(network: solana) {
    transfers(
      signature: {in: [""" + tx_signatures + """]}
    ) {
      instruction {
        action {
          name
          type
        }
        callPath
      }
      amount(success: {is: true})
      transaction {
        signer
        signature
      }
      block {
        timestamp {
          iso8601
        }
      }
      currency {
        name
        address
        symbol
        decimals
      }
      sender {
        address
        mintAccount
        type
      }
      receiver {
        address
        mintAccount
        type
      }
    }
  }
}
"""

result = run_query(query)
# convert GraphQL json to pandas dataframe
withdrawals = pd.json_normalize(result['data']['solana']['transfers'])

withdrawals

# def get_transfer_fields(x):
#     print(x)
#     lastrow = x.loc[x["instruction.action.name"]=="transfer"].sort_values("instructino.callPath")[-1] 
#     return lastrow[["amount", "currency.name", "receiver.mintAccount"]]
final = withdrawals.loc[withdrawals["instruction.action.name"]=="burn"].groupby("transaction.signature").last().reset_index()

final

,transaction.signature,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type
0,2RY7GFRgZmtFsV9yBVhZUj59knjH2vKcghZyYnYCKYEQmba7UrX2vJZCmrepBpJWARMv9iAq2CoK2rHmth1L6GZv,166.136589,burn,burn,1-1,HK7aLFrSXgUhaTPCJpEBDS6kgfwG9kJUBUqThFhX5PMG,2022-03-20T10:23:54Z,,EBPM7fvPN8EuA65Uc7DT9eGyDUZ1sqMLM8Rb8y2YxBYU,,6,HK7aLFrSXgUhaTPCJpEBDS6kgfwG9kJUBUqThFhX5PMG,8xmwVhJBSkC4GjZ5fBNK4qVCWdpvR8ws9NGbqg1AYwTW,account,,,
1,2TxGistAH6ju3EDJ7bFBARRTG2mCPiQfeAAPvr9XjENaQZ6UtTKZ76inec9HRCwwvmwtGnwKy3d2nh1qAAHkrSBg,9457.058295,burn,burn,1-1,B8jPR8uyjg3fQd6KbLEeFZNpSxhUiHVN83NyYxZHwSLm,2022-03-20T03:43:46Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,B8jPR8uyjg3fQd6KbLEeFZNpSxhUiHVN83NyYxZHwSLm,93T1cW6oY34fB8RkRtJWQYg6S45jDCCGbGGyf4LpSw1a,account,,,
2,2bryJhoSn3mTnxS2cJoNFwgaJdsSZiDs8gkoemEaGmGa2xt3KhFuEJKqmxbYGgoAXi78n8t4qjXc4ekcoWpEReHC,1.370946,burn,burn,3-1,2a1mgH9HKKfr75rVjyhj4KrYw8u6CKeXn25xL2PQ4G33,2022-03-20T10:55:47Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,2a1mgH9HKKfr75rVjyhj4KrYw8u6CKeXn25xL2PQ4G33,ErzouJLU9iK5AmKJwMVAAV8G4AneEGDLLyXeTc9VChu8,account,,,
3,2rTGu1KyXMtgpxXLKnpQu993X1uev8B2MiA29oiGDbBfeLHpcBfFpkz13Ax2prT9mxegrRgrSyvZitDPXKqmNCJR,2390.172640,burn,burn,1-1,CNFWTdU3MXz7jGLY6emWK1wNB6pMu84G5NrfqYmvMsYh,2022-03-20T05:05:37Z,,5SLqZSywodLS8ih6U2AAioZrxpgR149hR8SApmCB7r5X,,6,CNFWTdU3MXz7jGLY6emWK1wNB6pMu84G5NrfqYmvMsYh,EUFfeZozRq6sBE8U51nw5QCsj9v4qWMPCzGNxqGKkRn3,account,,,
4,2w7CWwPTGdYfpGGH283aaShxkmFgSh8jeU4FBCezZC5cZcYMS9Z7gxdwU8iHZ84hSA3YogzxfaJum3UfFtd3kZZn,0.932432,burn,burn,3-1,CfSV8111pAYdYHz4XQBnU6oKLFrkvnmTbfPZ3pANPxSo,2022-03-20T14:06:11Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,CfSV8111pAYdYHz4XQBnU6oKLFrkvnmTbfPZ3pANPxSo,5X7fxnKxyBmwh3jzbv4iMUXsNokPcugDiUATPm61oWx9,account,,,
5,2xVXVRsBfz8u1cRF6qPfwQfnYumt7SsY9fHq5omYmFVhYjmGqHB7U8h9TDKyta7GXrcUMZh9VEV1Gb7hTmH4ygab,65.348042,burn,burn,0-1,2cgrNDTMhgjdvUaHH1RyH6vji9MVPN53Sq1FLrAB9S78,2022-03-20T12:29:52Z,,EBPM7fvPN8EuA65Uc7DT9eGyDUZ1sqMLM8Rb8y2YxBYU,,6,2cgrNDTMhgjdvUaHH1RyH6vji9MVPN53Sq1FLrAB9S78,7TuiJMgdwSjj4sqVv5DCabprd79XTnzgSatdbkbEJcui,account,,,
6,2xWT5Hcb5jJAyqHoD162K3PEnf4BDCgRuCmARB5uyAn2wTp9rSpphTtVawu85bLp1RZPTgu8QyMgZirsXPzBjLgx,3006.208817,burn,burn,0-1,4SfZ3DSk2LnPvs5jxXJi7jcUvXWPY7xMs5UyS7JJHtRw,2022-03-20T10:39:21Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,4SfZ3DSk2LnPvs5jxXJi7jcUvXWPY7xMs5UyS7JJHtRw,9hdRmzHDyhftiWk96bjkvUyA95EyDgv1BqQeKfan86rb,account,,,
7,2xjsy3LjPFqY87Q9kgn4UPCuArHEgD3Z38MnKSxt5YReqAhgXaVGEmErvPDAhkHeVeP68mjATyTKKUr2Dw9rfsRL,0.913964,burn,burn,2-0,C5KamiY2oFw9pfBNMRZ9K2eq7dz5Rdp7dPxvZPH6JA2K,2022-03-20T06:21:32Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,C5KamiY2oFw9pfBNMRZ9K2eq7dz5Rdp7dPxvZPH6JA2K,93ZXrDirLmsdda8kqeTEowSS3nomFE25bB7C1koqEWSy,account,,,
8,36p2eDersDrmGEfKxT2aRa4Sqkju55ysK5geGQiFVFzE4zywCTiGRLVDwxyJe1Ki4smThboLeRu6PEXCW5fgE4bw,14.570184,burn,burn,3-1,CfSV8111pAYdYHz4XQBnU6oKLFrkvnmTbfPZ3pANPxSo,2022-03-20T14:10:48Z,,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,,9,CfSV8111pAYdYHz4XQBnU6oKLFrkvnmTbfPZ3pANPxSo,CVzxc5pcNB3R4a8sG1Jw2TP6zopBnM1okznpHxRrS1kA,account,,,
9,3KdYLEwbbBT9tA9ezP8o9h6RzR1822BBuSH4jMKANEStwDpQaajgFnYJe3niNS7FmcYHBy55NkraLeeDouf6idj3,760.964650,burn,burn,0-1,8LeeNzgWrhvRjYftzAjrycUFJvCXb9TLskSgj5z34JHL,2022-03-20T14:46:11Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,8LeeNzgWrhvRjYftzAjrycUFJvCXb9TLskSgj5z34JHL,CGbHW9iEFW6sdygZaPSoL9a3AxzEXVUDdarrZJYnfSqj,account,,,


In [355]:
final

,transaction.signature,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type
0,2RY7GFRgZmtFsV9yBVhZUj59knjH2vKcghZyYnYCKYEQmba7UrX2vJZCmrepBpJWARMv9iAq2CoK2rHmth1L6GZv,166.136589,burn,burn,1-1,HK7aLFrSXgUhaTPCJpEBDS6kgfwG9kJUBUqThFhX5PMG,2022-03-20T10:23:54Z,,EBPM7fvPN8EuA65Uc7DT9eGyDUZ1sqMLM8Rb8y2YxBYU,,6,HK7aLFrSXgUhaTPCJpEBDS6kgfwG9kJUBUqThFhX5PMG,8xmwVhJBSkC4GjZ5fBNK4qVCWdpvR8ws9NGbqg1AYwTW,account,,,
1,2TxGistAH6ju3EDJ7bFBARRTG2mCPiQfeAAPvr9XjENaQZ6UtTKZ76inec9HRCwwvmwtGnwKy3d2nh1qAAHkrSBg,9457.058295,burn,burn,1-1,B8jPR8uyjg3fQd6KbLEeFZNpSxhUiHVN83NyYxZHwSLm,2022-03-20T03:43:46Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,B8jPR8uyjg3fQd6KbLEeFZNpSxhUiHVN83NyYxZHwSLm,93T1cW6oY34fB8RkRtJWQYg6S45jDCCGbGGyf4LpSw1a,account,,,
2,2bryJhoSn3mTnxS2cJoNFwgaJdsSZiDs8gkoemEaGmGa2xt3KhFuEJKqmxbYGgoAXi78n8t4qjXc4ekcoWpEReHC,1.370946,burn,burn,3-1,2a1mgH9HKKfr75rVjyhj4KrYw8u6CKeXn25xL2PQ4G33,2022-03-20T10:55:47Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,2a1mgH9HKKfr75rVjyhj4KrYw8u6CKeXn25xL2PQ4G33,ErzouJLU9iK5AmKJwMVAAV8G4AneEGDLLyXeTc9VChu8,account,,,
3,2rTGu1KyXMtgpxXLKnpQu993X1uev8B2MiA29oiGDbBfeLHpcBfFpkz13Ax2prT9mxegrRgrSyvZitDPXKqmNCJR,2390.172640,burn,burn,1-1,CNFWTdU3MXz7jGLY6emWK1wNB6pMu84G5NrfqYmvMsYh,2022-03-20T05:05:37Z,,5SLqZSywodLS8ih6U2AAioZrxpgR149hR8SApmCB7r5X,,6,CNFWTdU3MXz7jGLY6emWK1wNB6pMu84G5NrfqYmvMsYh,EUFfeZozRq6sBE8U51nw5QCsj9v4qWMPCzGNxqGKkRn3,account,,,
4,2w7CWwPTGdYfpGGH283aaShxkmFgSh8jeU4FBCezZC5cZcYMS9Z7gxdwU8iHZ84hSA3YogzxfaJum3UfFtd3kZZn,0.932432,burn,burn,3-1,CfSV8111pAYdYHz4XQBnU6oKLFrkvnmTbfPZ3pANPxSo,2022-03-20T14:06:11Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,CfSV8111pAYdYHz4XQBnU6oKLFrkvnmTbfPZ3pANPxSo,5X7fxnKxyBmwh3jzbv4iMUXsNokPcugDiUATPm61oWx9,account,,,
5,2xVXVRsBfz8u1cRF6qPfwQfnYumt7SsY9fHq5omYmFVhYjmGqHB7U8h9TDKyta7GXrcUMZh9VEV1Gb7hTmH4ygab,65.348042,burn,burn,0-1,2cgrNDTMhgjdvUaHH1RyH6vji9MVPN53Sq1FLrAB9S78,2022-03-20T12:29:52Z,,EBPM7fvPN8EuA65Uc7DT9eGyDUZ1sqMLM8Rb8y2YxBYU,,6,2cgrNDTMhgjdvUaHH1RyH6vji9MVPN53Sq1FLrAB9S78,7TuiJMgdwSjj4sqVv5DCabprd79XTnzgSatdbkbEJcui,account,,,
6,2xWT5Hcb5jJAyqHoD162K3PEnf4BDCgRuCmARB5uyAn2wTp9rSpphTtVawu85bLp1RZPTgu8QyMgZirsXPzBjLgx,3006.208817,burn,burn,0-1,4SfZ3DSk2LnPvs5jxXJi7jcUvXWPY7xMs5UyS7JJHtRw,2022-03-20T10:39:21Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,4SfZ3DSk2LnPvs5jxXJi7jcUvXWPY7xMs5UyS7JJHtRw,9hdRmzHDyhftiWk96bjkvUyA95EyDgv1BqQeKfan86rb,account,,,
7,2xjsy3LjPFqY87Q9kgn4UPCuArHEgD3Z38MnKSxt5YReqAhgXaVGEmErvPDAhkHeVeP68mjATyTKKUr2Dw9rfsRL,0.913964,burn,burn,2-0,C5KamiY2oFw9pfBNMRZ9K2eq7dz5Rdp7dPxvZPH6JA2K,2022-03-20T06:21:32Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,C5KamiY2oFw9pfBNMRZ9K2eq7dz5Rdp7dPxvZPH6JA2K,93ZXrDirLmsdda8kqeTEowSS3nomFE25bB7C1koqEWSy,account,,,
8,36p2eDersDrmGEfKxT2aRa4Sqkju55ysK5geGQiFVFzE4zywCTiGRLVDwxyJe1Ki4smThboLeRu6PEXCW5fgE4bw,14.570184,burn,burn,3-1,CfSV8111pAYdYHz4XQBnU6oKLFrkvnmTbfPZ3pANPxSo,2022-03-20T14:10:48Z,,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,,9,CfSV8111pAYdYHz4XQBnU6oKLFrkvnmTbfPZ3pANPxSo,CVzxc5pcNB3R4a8sG1Jw2TP6zopBnM1okznpHxRrS1kA,account,,,
9,3KdYLEwbbBT9tA9ezP8o9h6RzR1822BBuSH4jMKANEStwDpQaajgFnYJe3niNS7FmcYHBy55NkraLeeDouf6idj3,760.964650,burn,burn,0-1,8LeeNzgWrhvRjYftzAjrycUFJvCXb9TLskSgj5z34JHL,2022-03-20T14:46:11Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,8LeeNzgWrhvRjYftzAjrycUFJvCXb9TLskSgj5z34JHL,CGbHW9iEFW6sdygZaPSoL9a3AxzEXVUDdarrZJYnfSqj,account,,,


# Cancel Pending Deposit

In [345]:
# Format this shit so it plugs into the query

tx_signatures = list(df.loc[
    df["log.instruction"]=="CancelPendingDeposit"]["transaction.signature"].iloc[:50].values)

tx_signatures = str(tx_signatures)[1:-1].replace("\'", "\"").replace("\n", "")

query = """
query MyQuery {
  solana(network: solana) {
    transfers(
      signature: {in: [""" + tx_signatures + """]}
    ) {
      instruction {
        action {
          name
          type
        }
        callPath
      }
      amount(success: {is: true})
      transaction {
        signer
        signature
      }
      block {
        timestamp {
          iso8601
        }
      }
      currency {
        name
        address
        symbol
        decimals
      }
      sender {
        address
        mintAccount
        type
      }
      receiver {
        address
        mintAccount
        type
      }
    }
  }
}
"""

result = run_query(query)
# convert GraphQL json to pandas dataframe
cancel_withdraw = pd.json_normalize(result['data']['solana']['transfers'])

cancel_withdraw

# def get_transfer_fields(x):
#     print(x)
#     lastrow = x.loc[x["instruction.action.name"]=="transfer"].sort_values("instructino.callPath")[-1] 
#     return lastrow[["amount", "currency.name", "receiver.mintAccount"]]
final = cancel_withdraw.loc[cancel_withdraw["instruction.action.name"]=="burn"].groupby("transaction.signature").last().reset_index()

final

,transaction.signature,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type


In [348]:
cancel_withdraw = cancel_withdraw.loc[cancel_withdraw["currency.name"]!="Wrapped SOL"].sort_values("instruction.callPath")

In [349]:
final

,transaction.signature,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type


In [352]:
# def get_transfer_fields(x):
#     print(x)
#     lastrow = x.loc[x["instruction.action.name"]=="transfer"].sort_values("instructino.callPath")[-1] 
#     return lastrow[["amount", "currency.name", "receiver.mintAccount"]]
final = cancel_withdraw.loc[cancel_withdraw["instruction.action.name"]=="transfer"].groupby("transaction.signature").last().reset_index()

In [353]:
final

,transaction.signature,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type
0,2B6KDjyqkvQ1oQMp9mDhw1BqWmfSsWNhZHXyNyUXchiXRd9xrJmwvj2X1hk9jeC4QvmNWMZD9wDZkfpmjWWoQoHQ,19.275855,transfer,transfer,0-0,E6gtYUmFLbfiEMyeHa9uQpoyS1bqH2Zi3SKYvTALkdvQ,2022-03-22T00:39:30Z,Marinade staked SOL (mSOL),mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,mSOL,9,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,Bm8ju1pbvvCqnMs2aC9BKrdUSVEdBLTBvAuMG34BVhDY,account,E6gtYUmFLbfiEMyeHa9uQpoyS1bqH2Zi3SKYvTALkdvQ,5hdehwzGgTHnU4MYyhG2cv7ETJwuEYoCpEFpP2wv1xjM,account
1,2KDb9JiffSw5gsu8ShxSH7ZeQb8u72eJCFYEP7GT1MnCFHxY6htVUxrY9pxkkF3ywVMfWHdnrmf2WPpXbJ4PMt24,35309.183369,transfer,transfer,0-0,6fJqGBoLuYxhkgKcXPwDHquZwCAsFXQdpAsGHdbZoYCL,2022-03-22T07:14:54Z,USD Coin,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,USDC,6,6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK,CSbP824yiNVevVsJVHwpov2AgCf2McCLiFyKGjUTd9qr,account,6fJqGBoLuYxhkgKcXPwDHquZwCAsFXQdpAsGHdbZoYCL,6ZWxbbRS7iGSVdZAGzwU4gCE8hJ73oxq49jQuEqrycB7,account
2,2TWwuYTpvGv6uTkNfnG244uuMbMqrcrhMm29gvquzPX3WnWdA3ixddHGbQxHK3Y6g39FjpwrGCeY459GqJEUDqnh,0.003350,transfer,transfer,0-0,GwYc9Uz9UUmikf9g276jd9SLQyQvEqSZc82m2vRp9rsr,2022-03-22T04:11:51Z,Wrapped Bitcoin (Sollet),9n4nbM75f5Ui33ZbPYXn59EwSgE8CGsHtAeTH5YFeJ9E,BTC,6,DA1M8mw7GnPNKU9ReANtHPQyuVzKZtsuuSbCyc2uX2du,3LWwNVeYGRspS6JTmcEWVoD3yVmVps9ELceq77pTv3jj,account,GwYc9Uz9UUmikf9g276jd9SLQyQvEqSZc82m2vRp9rsr,5KpnP5JtpvKpX89w5ckzqXWdhWAgvANQ9ePJTrxgCuUU,account
3,2UxPketrN9cmbDax2RArwbv1tr66dQnS4RVDSbXkv6NRjrXhATFDbheCto6tLiUEsyrYE2gT2E8BpWwR4zvZfeTf,11423.904399,transfer,transfer,0-0,BCwCpeABMuENfyX8C8dVW24fjByz1ZdoUM7ASYk8BnqE,2022-03-22T00:38:47Z,Saber Protocol Token,Saber2gLauYim4Mvftnrasomsv6NvAuncvMEZwcLpD1,SBR,6,BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h,45NeXayPBdfZYWcoZPdv3sQeXoYVS5GcHRWP4MHXBSHY,account,BCwCpeABMuENfyX8C8dVW24fjByz1ZdoUM7ASYk8BnqE,5Ttz29SShNtt7imK8KwtAxdKqjNWzbRhcW59wDidzao8,account
4,2bB2ma1GPE5r5B9gzG2Az5Mv1k6RSVKr5Z4ei4T7CvBsmhFF3YZ6mnb223J8MtNUtzp2EoP4FDs8Q5eXmm6T8C8M,300.000000,transfer,transfer,0-0,E9aDFqixBSSEVmHqCGwoJPdLozDG4Rbk6TRyoW55S7xf,2022-03-21T12:21:07Z,USD Coin,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,USDC,6,6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK,CSbP824yiNVevVsJVHwpov2AgCf2McCLiFyKGjUTd9qr,account,E9aDFqixBSSEVmHqCGwoJPdLozDG4Rbk6TRyoW55S7xf,6zRLgzEbrQNHejBh81AcJPgg613dnWvsDxrM39ceVxyZ,account
5,2bZhuytNSrfosSg3fB9dA9U7TccUV1sYNpP4ZworMcpdiUeXsu7dCWLjM7E9DsFCFG59uLGv9rKmamVtYa7AbacQ,145.332974,transfer,transfer,0-0,EzasAUgQXhbrVM5U4kVo8wTzBn1xzQn8aEv5a7HYNApp,2022-03-23T09:35:33Z,Marinade staked SOL (mSOL),mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,mSOL,9,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,Bm8ju1pbvvCqnMs2aC9BKrdUSVEdBLTBvAuMG34BVhDY,account,EzasAUgQXhbrVM5U4kVo8wTzBn1xzQn8aEv5a7HYNApp,3npcSN8spwZBkXfPG14RmvE3fdHVh7sD8Bs21MXVyK8n,account
6,2ceapecJGaptUp5cBGnCAFLVfyMUgKMALsNhnd8dSp6HnkeycZq3qoVXFFJs8de5gcZq6YhpuoWCZ3ivcFfFBVqE,1928.994718,transfer,transfer,0-0,EHWCVPPek2kt512pn6JQDKMdZ7R91gLad24dthVYjUQo,2022-03-20T11:57:40Z,UST (Wormhole),9vMJfxuKxXBoEa7rM12mYLMwTacLMLDJqHozw96WQL8i,UST,6,5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj,69Znv2Lj1XbaSWx5B8grnF7UKG1vG14B3XtKPGA6GrXP,account,EHWCVPPek2kt512pn6JQDKMdZ7R91gLad24dthVYjUQo,ES4De2T8ZG7E7EPRM9eSwFppbfKBioRnEUW7XRr7Uaju,account
7,2qaktx6pzjVc8B3eHkVLWYKmtmgLjXYg2G99Hxmgo1b9Bq1nr2S4JGk32AtUbaTXTpdwUWyaxWnTKHNArzH9La6A,16.007000,transfer,transfer,2-0,FxqCgkXBXFrcTdL7UiX28Uj6BKtuf6FV5p5EFAy2Vw6K,2022-03-23T00:30:55Z,Solana,-,SOL,9,HiepGSjE1udH3GDTzwHF1bdmNKyuTUiweWu1vsx8okPk,HiepGSjE1udH3GDTzwHF1bdmNKyuTUiweWu1vsx8okPk,account,DWdWFU1GECh3BkuXTY6rALwLPPoRtRWQe8UJ4axvhLiV,DWdWFU1GECh3BkuXTY6rALwLPPoRtRWQe8UJ4axvhLiV,account
8,2zw2vDUudwnTu6dhcigW1kzNGSoTv3JJWkcks3rK2mW4ks624tqdjSXYs73HVNJNiWLQYDQpgKm7YhEyiuaHRhC5,1935.9163

# Claim Withdrawal Demo

In [364]:
# Format this shit so it plugs into the query

tx_signatures = list(df.loc[
    df["log.instruction"]=="ClaimPendingWithdrawal"]["transaction.signature"].iloc[50:100].values)

tx_signatures = str(tx_signatures)[1:-1].replace("\'", "\"").replace("\n", "")

In [365]:
query = """
query MyQuery {
  solana(network: solana) {
    transfers(
      signature: {in: [""" + tx_signatures + """]}
    ) {
      instruction {
        action {
          name
          type
        }
        callPath
      }
      amount(success: {is: true})
      transaction {
        signer
        signature
      }
      block {
        timestamp {
          iso8601
        }
      }
      currency {
        name
        address
        symbol
        decimals
      }
      sender {
        address
        mintAccount
        type
      }
      receiver {
        address
        mintAccount
        type
      }
    }
  }
}
"""

result = run_query(query)
# convert GraphQL json to pandas dataframe
cancel_withdrawals = pd.json_normalize(result['data']['solana']['transfers'])

In [366]:
cancel_withdrawals = cancel_withdrawals.loc[cancel_withdrawals["currency.name"]!="Wrapped SOL"].sort_values("instruction.callPath")

In [367]:
cancel_withdrawals

,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,transaction.signature,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type
40,0.002039,createAccount,createAccount,0,CS36Qj6Z618rzkJsdfxKvarGEnnPZpja3A2pLUk3mtqg,a1UhAgVNNfYL9pCwe3o19tV4MbY2wWiT1K3DfiHRaq6AyZudB4WfUMojfxtteHLMtacAZrsovXf3U7PwZDwbvFu,2022-03-07T23:36:52Z,Solana,-,SOL,9,CS36Qj6Z618rzkJsdfxKvarGEnnPZpja3A2pLUk3mtqg,CS36Qj6Z618rzkJsdfxKvarGEnnPZpja3A2pLUk3mtqg,account,CWAh51cYwy9zikASvFzjw4bB1cFpriFnWkk2dPLLfth4,CWAh51cYwy9zikASvFzjw4bB1cFpriFnWkk2dPLLfth4,account
44,0.002039,createAccount,createAccount,0,Fapv1QjWPNtzEMjMEaDQ3V7vpwy7j6kPKVMafpZVbSMK,2EVcmW9P9JJ17E1b3j4WsJ1ebLeHvL4wADAhFnJTgsKsiW5q1i2xrnYgpgibZzfBkdMJpEBoAfBLjN9wdNg3WZKm,2022-03-07T08:58:12Z,Solana,-,SOL,9,Fapv1QjWPNtzEMjMEaDQ3V7vpwy7j6kPKVMafpZVbSMK,Fapv1QjWPNtzEMjMEaDQ3V7vpwy7j6kPKVMafpZVbSMK,account,86M27mMNXiXQQcVsRChLa9e3vbXLTVJ4GZ26awCDdUw5,86M27mMNXiXQQcVsRChLa9e3vbXLTVJ4GZ26awCDdUw5,account
43,0.002039,createAccount,createAccount,0,EWJ67eEnhLXecHGSyncgdb2mjaX42FzE3yA8HPcCn1T4,4kzPCLmVm7PXc2v89LDHqan7RrpzcpMNWgeAocr9MubFXLKukMwaPbLEsjrKuX1JFFjpxDugT73oJJggjtz6tJc8,2022-03-08T05:36:21Z,Solana,-,SOL,9,EWJ67eEnhLXecHGSyncgdb2mjaX42FzE3yA8HPcCn1T4,EWJ67eEnhLXecHGSyncgdb2mjaX42FzE3yA8HPcCn1T4,account,BboxR2YPw74fdLQNqFAXbPsoTqpmv5qGUpJD3iEAwhc9,BboxR2YPw74fdLQNqFAXbPsoTqpmv5qGUpJD3iEAwhc9,account
42,0.002039,createAccount,createAccount,0,E3KVdguWpnEdoe9emdfq6e1VnWwFa4UWffQk8imbrWeq,CaJf5VoBmV8bh7KCnB8Ta5keVML56Yh6Vtaj3K87SDRMsf82947ZhHdWSyt9amULbQhUu7YUHxcpzwt1LAjbtob,2022-03-07T23:26:25Z,Solana,-,SOL,9,E3KVdguWpnEdoe9emdfq6e1VnWwFa4UWffQk8imbrWeq,E3KVdguWpnEdoe9emdfq6e1VnWwFa4UWffQk8imbrWeq,account,7rcC87RhNoeZiPH6wCw3Dw5CmMgQoYoygfGBvXzy5M6C,7rcC87RhNoeZiPH6wCw3Dw5CmMgQoYoygfGBvXzy5M6C,account
41,0.002039,createAccount,createAccount,0,CsKGnnpYiMVa59a5ygk3FfqibfKS7TJcPHGsHDMPYchD,2mv2xmAvQpGq5RmrbpDjA4fTiXZrKFRK1pEkrWhSY9bKC1hFMf5ANxrFmxmTGHfvnnsjpc7ou628um9VXzRjNgVE,2022-03-07T01:56:35Z,Solana,-,SOL,9,CsKGnnpYiMVa59a5ygk3FfqibfKS7TJcPHGsHDMPYchD,CsKGnnpYiMVa59a5ygk3FfqibfKS7TJcPHGsHDMPYchD,account,9U7YsJMnAR32BLZMyEE7AWoUCZboEfbuahZ3HJVLAyz9,9U7YsJMnAR32BLZMyEE7AWoUCZboEfbuahZ3HJVLAyz9,account
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,297.426170,closeAccount,closeAccount,3,GcWZxE1mgDSpqnzBJtTjVfRvyKweWD6RnwuPyzPdsQiC,v5hwTun1u9gZfCSXkPMN5G5XRGNmrwhXE4sBZauuWcUpES2ixQxWTQXxq4zSXfSBuhaPxKB6VzcnJs5w52MD3qS,2022-03-07T22:29:52Z,Solana,-,SOL,9,5VUH7LjDzPCPVYtYSCtuGWVwEXPDcB29jzimpehMD6Y2,5VUH7LjDzPCPVYtYSCtuGWVwEXPDcB29jzimpehMD6Y2,account,GcWZxE1mgDSpqnzBJtTjVfRvyKweWD6RnwuPyzPdsQiC,GcWZxE1mgDSpqnzBJtTjVfRvyKweWD6RnwuPyzPdsQiC,account
27,31.092180,closeAccount,closeAccount,3,HHxvyECtG2WxfQmhEfF7nGmY5UpqZwKBB2iFDmDxk8Z6,2pAvSC3hTjYFbPCZzusSrKg1N6Ewh6AtER56DTfzeyejzZFxsHFHYrsAcfvzWDzFZviBmENVgq3Luq4RT8jrBcsh,2022-03-07T02:34:57Z,Solana,-,SOL,9,6Vz5C31vctLMfKJJwqkvWmkCznYdVjXMp8iu43xXbL3b,6Vz5C31vctLMfKJJwqkvWmkCznYdVjXMp8iu43xXbL3b,account,HHxvyECtG2WxfQmhEfF7nGmY5UpqZwKBB2iFDmDxk8Z6,HHxvyECtG2WxfQmhEfF7nGmY5UpqZwKBB2iFDmDxk8Z6,account
29,1.631163,closeAccount,closeAccount,3,J2xjATP9kNpXCFcJacw66neLKfH1f3LNz5DY2uT7MRcJ,5pZVktXDwcMDS49c22q3qiZ2MnUpYM9HC6AYhqmXz6emZpPkYYyGx8EXFNv7j6XMLfwG2inZFYhFjZLoQVjVXPbH,2022-03-07T10:30:23Z,Solana,-,SOL,9,Ddw9kR7pzhq7mN9ufxonkwoadDLgJCBaKrSi81PQPj5t,Ddw9kR7pzhq7mN9ufxonkwoadDLgJCBaKrSi81PQPj5t,account,J2xjATP9kNpXCFcJacw66neLKfH1f3LNz5DY2uT7MRcJ,J2xjATP9kNpXCFcJacw66neLKfH1f3LNz5DY2uT7MRcJ,account
3,20.069807,closeAccount,closeAccount,3,2iFQRqweHHoFZUHU2m1wCfrpGrmcUe2YmAPRsLpMLTxV,2gPr8VGozSbXtn64i8h6fYdA4aJbfbTGo99Urh1S9JckUcTBKfvhz4w8SPYvSm4avrGqZezX4T2jv1GFmkVe8APe,2022-03-07T06:04:20Z,Solana,-,SOL,9,75AFGaXoh2UtCVeR5JBJEEKKHWGxG78xEqAP8od91nEi,75AFGaXoh2UtCVeR5JBJEEKKHWGxG78xEqAP8od91nEi,account,2iFQRqweHHoFZUHU2m1wCfrpGrmcUe2YmAPRsLpMLTxV,2iFQRqweHHoF

In [368]:
# def get_transfer_fields(x):
#     print(x)
#     lastrow = x.loc[x["instruction.action.name"]=="transfer"].sort_values("instructino.callPath")[-1] 
#     return lastrow[["amount", "currency.name", "receiver.mintAccount"]]
final = cancel_withdrawals.loc[cancel_withdrawals["instruction.action.name"]=="mintTo"].groupby("transaction.signature").last().reset_index()

In [324]:
final

,transaction.signature,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type
0,3UTfRTMFkCd3hz5UpZCp5JhM4SY8raum8yJQJAxErj7UwEbKRNzifSY7F2XdiuZihYVK9mi9hFKtWmmMUMGtA7Cx,0.278102,mintTo,mintTo,0-0,Eu8S7Kt3MtfSDx4UsNh6FaBw8j5DYhC53L48f8D9R4c5,2022-03-27T20:54:10Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,,,,Eu8S7Kt3MtfSDx4UsNh6FaBw8j5DYhC53L48f8D9R4c5,EBktJULvmhnng2zwkUwyjPqejPzxbJjyHqey7nMwQLzy,account
1,3kKBErted5MM9C6yRkTCLFk4TL8eDCuAZybGsLUNgaog7S2F5xasbiaeMNYU4eQ1fqzva71bwiGyJQhJKQuL1qDS,4.563915,mintTo,mintTo,0-0,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,2022-03-28T17:11:28Z,,5SLqZSywodLS8ih6U2AAioZrxpgR149hR8SApmCB7r5X,,6,,,,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,4ZUqHFxi7sP6BpPa28hvK8bqrk4DQJEktUzdymkySvfq,account
2,3nCnNMb3q9j67HnpyG7ZKEd2AADMRK2n8FvysGcrMMSy5mSTsAZSGN5quwgYPsyqg2E1DnbRxEVWUcTQh8W73e6e,6.702134,mintTo,mintTo,0-0,8cSDdsAe4kBxfTD28H2HRzT7Hhp5DZxihRZGL2Lpmwrt,2022-03-28T11:37:57Z,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,,9,,,,8cSDdsAe4kBxfTD28H2HRzT7Hhp5DZxihRZGL2Lpmwrt,5NjpSprzTR3c1WiLHaCUrbYvP1wKykL5sMveUc9ALo29,account
3,45NPN2TTJ8FduLkwFASuebq5Jrvs2NgyR3t8KHtk4TMMj1DBJpePbEo5nFoszGetgTPTKKLSbTPM3vhZzEcFr4eG,29.507184,mintTo,mintTo,0-0,EgzRJHeqbfYsCL4VBZCsq2ZEpk5M7jzWc4n2N53SyFKJ,2022-03-28T14:24:47Z,,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,,6,,,,EgzRJHeqbfYsCL4VBZCsq2ZEpk5M7jzWc4n2N53SyFKJ,DYod4eQKxy4XThto93MNGng7tXv9p7Rgyn6MDNNCg8kW,account
4,4Aa1mQ119X8epmyp3jNYFN4mgKJGrR1b7V9B3g29TkLBvukmAQo3N6jCUscj1YYfR4ZFRXcWjYhoTx8VTz5NJJz8,4805.700237,mintTo,mintTo,0-0,42i4VhjfjDv1DYdSfGsZXW53eGqFhBdWU6Lg6FcoPNg8,2022-03-28T09:46:58Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,,,,42i4VhjfjDv1DYdSfGsZXW53eGqFhBdWU6Lg6FcoPNg8,D2aJY1GDWhVzZQitbWUbWzsvc5oVvXr371CdUJiJnHKu,account
5,5THSMzhv6NAkNRLSY2LDAfAE8nEx7rycV5QnEpKMs1TkjpVN5jvwGU5NRQgcHHVorFF4MRtSDq673HpGaDQZQC9x,265.313148,mintTo,mintTo,0-0,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,2022-03-27T16:54:27Z,,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,,9,,,,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,FzBVYYkYRtsWWSTNmxdrx8iKZdCcBUGnMGsoowg6JCvJ,account
6,ENWyWi7mjDon7AjAeDnEAB9zXwceEVdE2v53J7wN7ANQrcpBzkDMdWLhhHXApqxxUgBU6sTRciuSXVAqitp7tDk,5204.693398,mintTo,mintTo,0-0,BiEXHzcCgTgMLEPWVMJqCT38z1V1Xo3oxc7qxp2bUDUZ,2022-03-27T21:48:16Z,,DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE,,6,,,,BiEXHzcCgTgMLEPWVMJqCT38z1V1Xo3oxc7qxp2bUDUZ,EHsEiE5U4gMmPc5NEGtx1jqW4crWYvKKrzYY1JKjf4nw,account
7,VFA1SJ2iV96njG8RdpeL9eMe78kM3ju5MAsnDDCqpYZwdNuSwMLw8EZv7pN4DZouGzWpVX4KFfBTepGcM6KJhqc,3.713803,mintTo,mintTo,0-0,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,2022-03-27T16:59:18Z,,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,,9,,,,1wTvS4pFWQ2E8qht6eMuMhyqffRVLxPufdFkV9wARBR,FzBVYYkYRtsWWSTNmxdrx8iKZdCcBUGnMGsoowg6JCvJ,account
8,hx2LKHaY9CSr6B4ayYKLFiyjZj2isjRfBjTzfN1nEwZrqEbRW7DqM9q8aHYTjQ6mUceeG45AskXtzuDABDTqYzV,277.191034,mintTo,mintTo,0-0,7etxrP9LU9xwDVfGpEUrG1XYhq64ttE5rbrGhGymQrvY,2022-03-28T10:49:01Z,,Fw8mw9zNyfngvBYuS2L6XWfvFE8fExQ3apZVw4Ery5oy,,6,,,,7etxrP9LU9xwDVfGpEUrG1XYhq64ttE5rbrGhGymQrvY,3UXf3Sb1fVMhNCPNgTbn2uGL37iY2EryHdqhuEP7cjKv,account
9,pH5pnbCqa29bTiYuq2AQxaMyzzJZttJkQFunHGh1KKKJWq2YxfVXLyyHPnyysvUfRKBo311QLj4YVT9AwT9KTSZ,3.223691,mintTo,mintTo,0-0,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,2022-03-28T17:11:22Z,,SoAnGsHVqSyaN4MjWoPCcftC1V6oSeNxLJjF5TXrtuL,,6,,,,E1XDHsgM4MV7WfjbRCMC6ov1wtQrhdaQkfSobN4AQkSR,ED9iWMN5fiw3Cnon46fEvZaw2CqhDpFgw5Pnyu1qvN3T,account


# Withdrawal Demo

In [369]:
ix = pd.read_csv("friktion_ix.csv")

In [371]:
ix.columns

Index(['timestamp', 'txSignature', 'txSigner', 'instructionData',
       'instructionType'],
      dtype='object')